### Foundations of Computer Science 
#### Jacopo Roberto Nicosia
#### Matricola 861812

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
file1 = r"C:\Users\jacop\Desktop\Data Science\1° Semestre\Fundations of CS\Progetto\additional-kiva-snapshot\loans_lenders.csv"
file2 = r"C:\Users\jacop\Desktop\Data Science\1° Semestre\Fundations of CS\Progetto\additional-kiva-snapshot\loans.csv"
file3 = r"C:\Users\jacop\Desktop\Data Science\1° Semestre\Fundations of CS\Progetto\additional-kiva-snapshot\lenders.csv"
file4 = r"C:\Users\jacop\Desktop\Data Science\1° Semestre\Fundations of CS\Progetto\additional-kiva-snapshot\country_stats.csv"

In [3]:
loans_lenders = pd.read_csv(file1)
loans = pd.read_csv(file2, parse_dates=["disburse_time", "planned_expiration_time"])
lenders = pd.read_csv(file3)
country_stat = pd.read_csv(file4)

#### 1° Point
#### Normalize the loans_lenders table.
#### ( Each row must have one "loan_id" and one "lender")

In [4]:
loans_lenders_norm = loans_lenders.assign(lenders=loans_lenders.lenders.str.split(',')).explode('lenders').reset_index(drop=True)

In [5]:
loans_lenders_norm.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


#### 2° Point
#### For each loan add a column "duration" corrisponding to the number of days between "disburse time" and "planned expiration time".
#### If any of those dates is missing also the duration must be missing.

In [6]:
# this allows me to have only year/month/day in datetime format
loans['disburse_time'] = pd.to_datetime(loans['disburse_time'].dt.date) 
loans['planned_expiration_time'] = pd.to_datetime(loans['planned_expiration_time'].dt.date)

In [7]:
loans_duration = loans[['loan_id', 'disburse_time','planned_expiration_time']]

In [8]:
# Some 'planned_expiration_time' occurs before 'disburse_time'
# Fixing the problem by switching the dates 

In [9]:
temp = loans_duration.loc[loans_duration['disburse_time'] > loans_duration['planned_expiration_time']]

In [10]:
temp1 = temp.rename(columns = {'disburse_time':'planned_expiration_time', 'planned_expiration_time':'disburse_time'})

In [11]:
loans_duration.update(temp1)

C:\Users\jacop\Anaconda3\lib\site-packages\pandas\core\frame.py:5819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


In [12]:
loans_duration = loans_duration.assign(duration = loans_duration['planned_expiration_time'] - loans_duration['disburse_time'])

In [13]:
loans_duration.head()

,loan_id,disburse_time,planned_expiration_time,duration
0,657307.0,2013-12-22,2014-02-14,54 days
1,657259.0,2013-12-20,2014-03-26,96 days
2,658010.0,2014-01-09,2014-02-15,37 days
3,659347.0,2014-01-17,2014-02-21,35 days
4,656933.0,2013-12-17,2014-02-13,58 days


In [14]:
# Also checking the second condition of the assignment : "If any of those dates is missing also the duration must be missing."

In [15]:
loans_duration[loans_duration.duration.isnull()].head()

,loan_id,disburse_time,planned_expiration_time,duration
64,251406.0,2010-11-18,NaT,NaT
65,252537.0,2010-11-25,NaT,NaT
66,253044.0,2010-11-25,NaT,NaT
67,254012.0,2010-11-16,NaT,NaT
69,251336.0,2010-11-10,NaT,NaT


#### 3° Point
#### Find the lenders that have funded at least twice.

In [16]:
loans_status = loans[['loan_id', 'status']]

In [17]:
loans_merged = pd.merge(loans_lenders_norm, loans_status, on = 'loan_id', how = 'left')

In [18]:
# Dropping null values from 'status' 
loans_merged = loans_merged.dropna(subset=['status'])

In [19]:
loans_funded = loans_merged.loc[loans_merged['status'] == 'funded']

In [20]:
count = loans_funded['lenders'].value_counts()

In [21]:
# Storing the output of the count in a better way, renaming the columns for what they are
funded_twice = count[count>1].rename_axis('lenders').reset_index(name = 'number_of_loans')

In [22]:
funded_twice.head()

,lenders,number_of_loans
0,gooddogg1,142349
1,trolltech4460,139191
2,gmct,122156
3,nms,93283
4,themissionbeltco,70151


#### 4° Point
#### For each country, compute how many loans have involved that country as borrowers.

In [23]:
country = loans[['loan_id', 'country_code']]

In [24]:
c_count = country['country_code'].value_counts()

In [25]:
country_count = c_count.rename_axis('country_code').reset_index(name = 'number_of_loans')

In [26]:
country_count.head()

,country_code,number_of_loans
0,PH,285336
1,KE,143699
2,PE,86000
3,KH,79701
4,SV,64037


#### 5° Point
#### For each country, compute the overall amount of money borrowed.

In [27]:
# By the word "overall" I suppose that I don't have to consider the status of the loan
# If not so, then I would have included in the DF the column for status and dropped all the loans not 'funded'

country_borrow = loans[['loan_id', 'country_code', 'loan_amount']]

In [28]:
country_borrow1 = country_borrow.groupby('country_code').sum()['loan_amount']

In [29]:
country_borrow1 = country_borrow1.rename_axis('country_code').reset_index(name = 'money_borrowed')

In [30]:
country_borrow1.head()

,country_code,money_borrowed
0,AF,1967950.0
1,AL,4307350.0
2,AM,22950475.0
3,AZ,14784625.0
4,BA,477250.0


#### 6° Point
#### Like the previous point, but expressed as a percentage of the overall amount lent.

In [31]:
tot_amount = country_borrow1['money_borrowed'].sum()

In [32]:
country_borrow1['percentage'] = (country_borrow1['money_borrowed'] / tot_amount) * 100

In [33]:
country_borrow1.head()

,country_code,money_borrowed,percentage
0,AF,1967950.0,0.166578
1,AL,4307350.0,0.364598
2,AM,22950475.0,1.942654
3,AZ,14784625.0,1.251452
4,BA,477250.0,0.040397


#### 7° Point
#### Like the three previous points, but split for each year (with respect to disburse time).

In [34]:
country_per_year = loans[['loan_id','country_code', 'loan_amount', 'disburse_time']]

In [37]:
country_per_year['disburse_time'] = country_per_year['disburse_time'].dt.year
# here i need only the year for groupby purposes

C:\Users\jacop\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
country_per_year1 = pd.DataFrame(country_per_year.groupby(['country_code', 'disburse_time']).sum()['loan_amount'])

In [39]:
country_per_year1.head()

loan_amount
country_code disburse_time             
AF           2007.0            194975.0
             2008.0            365375.0
             2009.0            585125.0
             2010.0            563350.0
             2011.0            245125.0

#### 8° Point
#### For each lender, compute the overall amount of money lent. For each loan that has more than one lender, you must assume that all lenders contributed the same amount.

In [40]:
lenders_per_loan = loans_lenders_norm['loan_id'].value_counts() 
# since in the normalized_df there is a row for each lender associated to the loan_id i can use value count for knowing how many there are

In [41]:
lenders_per_loan_df = lenders_per_loan.rename_axis('loan_id').reset_index(name = 'number_of_lenders')

In [42]:
lenders_per_loan_df.head()

,loan_id,number_of_lenders
0,1433600,2888
1,1396525,2828
2,722883,2674
3,1398161,2619
4,1359159,2344


In [43]:
temp1 = pd.merge(loans_lenders_norm, loans[['loan_id','loan_amount']], on = 'loan_id')

In [44]:
temp2 = pd.merge(temp1, lenders_per_loan_df, on = 'loan_id')

In [45]:
temp2['fraction_per_loan'] = temp2['loan_amount'] / temp2['number_of_lenders']

In [46]:
amount_for_lender = pd.DataFrame(temp2.groupby('lenders').sum()['fraction_per_loan'])

In [47]:
amount_for_lender.head()

,fraction_per_loan
lenders,
000,1672.618411
00000,1380.693644
0002,2472.563566
00mike00,52.631579
0101craign0101,2623.565117


#### 9° Point
#### For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.

In [48]:
lenders_nationality = lenders[['permanent_name','country_code']]

In [ ]:
# first I get the distribution among the country that I have

In [49]:
prop1 = lenders_nationality['country_code'].value_counts()

In [50]:
prop2 = prop1.rename_axis('country_code').reset_index(name = 'count')

In [51]:
tot = prop2['count'].sum()

In [52]:
prop2['percentage'] = (prop2['count'] / tot) * 100

In [53]:
prop2['percentage'] /= prop2['percentage'].sum()
# In this way the 'percentage column' sum is equal to 1

In [54]:
filling_nation = lenders_nationality.loc[lenders_nationality['country_code'].isnull()]

In [55]:
# in this way I randomly fill the column (which is full of null) but mantaining the same distribution I got on prop2['percentage']
np.random.seed(1234)
filling_nation['country_code'] = np.random.choice(prop2['country_code'], size = len(filling_nation.index), p = prop2['percentage'])

C:\Users\jacop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [56]:
# Now I have the nationalities of each lender
filling_nation.head()

,permanent_name,country_code
0,qian3013,US
1,reena6733,US
2,mai5982,US
3,andrew86079135,AU
4,nguyen6962,GB


In [57]:
lenders_nationality.update(filling_nation)

In [58]:
# now joining to get what I need in the same df

In [59]:
lenders_nationality = lenders_nationality.rename(columns = {'permanent_name':'lenders'})

In [60]:
asd = pd.merge(loans_lenders_norm, lenders_nationality, on = 'lenders')

In [61]:
asdf = pd.merge(asd, country_borrow, on = 'loan_id') # country_borrow at point 5

In [62]:
asdf = asdf.rename(columns = {'country_code_x':'lender_country', 'country_code_y':'borrower_country'})

In [63]:
cash_outflows = asdf.groupby('lender_country').sum()['loan_amount']

In [64]:
# now merging with solution of point 5 to save time

In [65]:
trade_balance = pd.merge(country_borrow1, cash_outflows, left_on=['country_code'], right_on=['lender_country'])

In [66]:
trade_balance['difference'] = trade_balance['loan_amount'] - trade_balance['money_borrowed'] 

In [67]:
trade_balance.head() # negative -> cash inflows

,country_code,money_borrowed,percentage,loan_amount,difference
0,AF,1967950.0,0.166578,170800.0,-1797150.0
1,AL,4307350.0,0.364598,33850.0,-4273500.0
2,AM,22950475.0,1.942654,42075.0,-22908400.0
3,AZ,14784625.0,1.251452,11925.0,-14772700.0
4,BA,477250.0,0.040397,93075.0,-384175.0


#### 10° Point
#### Which country has the highest ratio between the difference computed at the previous point and the population?

In [68]:
country_stat1 = country_stat[['country_code','population','population_below_poverty_line']]

In [69]:
ratio = pd.merge(country_stat1, trade_balance[['country_code','difference']], on = 'country_code')

In [70]:
ratio['ratio_norm'] = abs(ratio['difference'] / ratio['population']) # Since it's not clear what does it mean "Highest" I'll use absolute value

In [71]:
ratio.loc[ratio['ratio_norm'] == ratio['ratio_norm'].max()]

,country_code,population,population_below_poverty_line,difference,ratio_norm
84,WS,196440,NaN,-12211000.0,62.161474


#### 11° Point
#### Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?

In [72]:
population_below_poverty_line_abs = ratio['population'] * (1 - (ratio['population_below_poverty_line'] / 100))

In [73]:
ratio['ratio_above'] = abs(ratio['difference'] / population_below_poverty_line_abs)

In [74]:
ratio.loc[ratio['ratio_above'] == ratio['ratio_above'].max()]

,country_code,population,population_below_poverty_line,difference,ratio_norm,ratio_above
68,AM,2930450,32.0,-22908400.0,7.817366,11.496126


#### 12° Point
#### For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. 
#### For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.

In [75]:
final_subset = pd.merge(loans_duration, loans[['loan_id','loan_amount']], on = 'loan_id').dropna()

In [76]:
final_subset.head()

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount
0,657307.0,2013-12-22,2014-02-14,54 days,125.0
1,657259.0,2013-12-20,2014-03-26,96 days,400.0
2,658010.0,2014-01-09,2014-02-15,37 days,400.0
3,659347.0,2014-01-17,2014-02-21,35 days,625.0
4,656933.0,2013-12-17,2014-02-13,58 days,425.0


In [ ]:
# creating 2 subset: 
# one in which i have all the loans that are disbursed and expired in the same year
# another one in which the loans last multiple years 
# -> in this latter one it's needed a solution to impute the quantity of money per year correctly 

In [77]:
same_year_loan = final_subset.loc[final_subset['disburse_time'].dt.year == final_subset['planned_expiration_time'].dt.year]

In [78]:
multiple_year_loan = final_subset.loc[final_subset['disburse_time'].dt.year < final_subset['planned_expiration_time'].dt.year]

In [79]:
multiple_year_loan.head()

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount
0,657307.0,2013-12-22,2014-02-14,54 days,125.0
1,657259.0,2013-12-20,2014-03-26,96 days,400.0
4,656933.0,2013-12-17,2014-02-13,58 days,425.0
12,660363.0,2013-12-23,2014-02-21,60 days,1175.0
13,661165.0,2013-12-26,2014-03-26,90 days,300.0


In [80]:
# creating a new row for each year between disburse_time and planned_expiration_time
multiple_year_loan = multiple_year_loan.loc[multiple_year_loan.index.repeat(multiple_year_loan.planned_expiration_time.dt.year - multiple_year_loan.disburse_time.dt.year + 1)]

In [ ]:
# now i want two columns needed for a cycle : i need a column for the beginning of the year and one for the end

In [81]:
# creating a new column in which i have the first day of the year, but with the 'disburse_time' year imputed
multiple_year_loan["start_time"] = multiple_year_loan.drop_duplicates()["disburse_time"].dt.year.apply(lambda x: dt.datetime(x, 1, 1))
multiple_year_loan.head()

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,start_time
0,657307.0,2013-12-22,2014-02-14,54 days,125.0,2013-01-01
0,657307.0,2013-12-22,2014-02-14,54 days,125.0,2013-01-01
1,657259.0,2013-12-20,2014-03-26,96 days,400.0,2013-01-01
1,657259.0,2013-12-20,2014-03-26,96 days,400.0,2013-01-01
4,656933.0,2013-12-17,2014-02-13,58 days,425.0,2013-01-01


In [82]:
# now fixing the year in start_time, in this way the year gets updated for each row of the same loan
y = multiple_year_loan["disburse_time"].dt.year
multiple_year_loan["start_time"] = pd.to_datetime(multiple_year_loan.groupby(multiple_year_loan["loan_id"]).cumcount() + y, format='%Y')
multiple_year_loan

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,start_time
0,657307.0,2013-12-22,2014-02-14,54 days,125.0,2013-01-01
0,657307.0,2013-12-22,2014-02-14,54 days,125.0,2014-01-01
1,657259.0,2013-12-20,2014-03-26,96 days,400.0,2013-01-01
1,657259.0,2013-12-20,2014-03-26,96 days,400.0,2014-01-01
4,656933.0,2013-12-17,2014-02-13,58 days,425.0,2013-01-01
...,...,...,...,...,...,...
1419604,989109.0,2015-11-13,2016-01-03,51 days,2425.0,2016-01-01
1419605,989143.0,2015-11-03,2016-01-05,63 days,100.0,2015-01-01
1419605,989143.0,2015-11-03,2016-01-05,63 days,100.0,2016-01-01
1419606,989240.0,2015-11-03,2016-01-03,61 days,175.0,2015-01-01


In [83]:
# doing the same thing already done with start time but computing the end of the year
multiple_year_loan["end_time"] = pd.to_datetime(multiple_year_loan.groupby(multiple_year_loan["loan_id"]).cumcount() + y, format='%Y')
multiple_year_loan.head()

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,start_time,end_time
0,657307.0,2013-12-22,2014-02-14,54 days,125.0,2013-01-01,2013-01-01
0,657307.0,2013-12-22,2014-02-14,54 days,125.0,2014-01-01,2014-01-01
1,657259.0,2013-12-20,2014-03-26,96 days,400.0,2013-01-01,2013-01-01
1,657259.0,2013-12-20,2014-03-26,96 days,400.0,2014-01-01,2014-01-01
4,656933.0,2013-12-17,2014-02-13,58 days,425.0,2013-01-01,2013-01-01


In [84]:
multiple_year_loan["end_time"] = multiple_year_loan["start_time"].dt.year.apply(lambda x: dt.datetime(x, 12, 31))
multiple_year_loan.head()

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,start_time,end_time
0,657307.0,2013-12-22,2014-02-14,54 days,125.0,2013-01-01,2013-12-31
0,657307.0,2013-12-22,2014-02-14,54 days,125.0,2014-01-01,2014-12-31
1,657259.0,2013-12-20,2014-03-26,96 days,400.0,2013-01-01,2013-12-31
1,657259.0,2013-12-20,2014-03-26,96 days,400.0,2014-01-01,2014-12-31
4,656933.0,2013-12-17,2014-02-13,58 days,425.0,2013-01-01,2013-12-31


In [85]:
# cycle to fill how many days of interest fall into the year 
def normalise_row(row):
    if row['disburse_time'].year == row['start_time'].year:
        return row['end_time'] - row['disburse_time']
    elif row['planned_expiration_time'].year == row['end_time'].year:
        return row['planned_expiration_time'] - row['start_time']
    else:
        return 365

In [86]:
multiple_year_loan['days'] = multiple_year_loan.apply(lambda x : normalise_row(x), axis=1) 

In [87]:
multiple_year_loan

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,start_time,end_time,days
0,657307.0,2013-12-22,2014-02-14,54 days,125.0,2013-01-01,2013-12-31,9 days 00:00:00
0,657307.0,2013-12-22,2014-02-14,54 days,125.0,2014-01-01,2014-12-31,44 days 00:00:00
1,657259.0,2013-12-20,2014-03-26,96 days,400.0,2013-01-01,2013-12-31,11 days 00:00:00
1,657259.0,2013-12-20,2014-03-26,96 days,400.0,2014-01-01,2014-12-31,84 days 00:00:00
4,656933.0,2013-12-17,2014-02-13,58 days,425.0,2013-01-01,2013-12-31,14 days 00:00:00
...,...,...,...,...,...,...,...,...
1419604,989109.0,2015-11-13,2016-01-03,51 days,2425.0,2016-01-01,2016-12-31,2 days 00:00:00
1419605,989143.0,2015-11-03,2016-01-05,63 days,100.0,2015-01-01,2015-12-31,58 days 00:00:00
1419605,989143.0,2015-11-03,2016-01-05,63 days,100.0,2016-01-01,2016-12-31,4 days 00:00:00
1419606,989240.0,2015-11-03,2016-01-03,61 days,175.0,2015-01-01,2015-12-31,58 days 00:00:00


In [88]:
# now changing the format -> needed for following operation
multiple_year_loan['days'] = pd.to_timedelta(multiple_year_loan['days'])

In [89]:
multiple_year_loan['days'] = multiple_year_loan['days'].dt.days.astype('int16')

In [90]:
multiple_year_loan['duration'] = multiple_year_loan['duration'].dt.days.astype('int16')

In [91]:
multiple_year_loan.head()

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,start_time,end_time,days
0,657307.0,2013-12-22,2014-02-14,54,125.0,2013-01-01,2013-12-31,9
0,657307.0,2013-12-22,2014-02-14,54,125.0,2014-01-01,2014-12-31,44
1,657259.0,2013-12-20,2014-03-26,96,400.0,2013-01-01,2013-12-31,11
1,657259.0,2013-12-20,2014-03-26,96,400.0,2014-01-01,2014-12-31,84
4,656933.0,2013-12-17,2014-02-13,58,425.0,2013-01-01,2013-12-31,14


In [92]:
multiple_year_loan['fraction_per_year'] = (multiple_year_loan['loan_amount']/multiple_year_loan['duration']) * multiple_year_loan['days']

In [93]:
multiple_year_loan.head()

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount,start_time,end_time,days,fraction_per_year
0,657307.0,2013-12-22,2014-02-14,54,125.0,2013-01-01,2013-12-31,9,20.833333
0,657307.0,2013-12-22,2014-02-14,54,125.0,2014-01-01,2014-12-31,44,101.851852
1,657259.0,2013-12-20,2014-03-26,96,400.0,2013-01-01,2013-12-31,11,45.833333
1,657259.0,2013-12-20,2014-03-26,96,400.0,2014-01-01,2014-12-31,84,350.000000
4,656933.0,2013-12-17,2014-02-13,58,425.0,2013-01-01,2013-12-31,14,102.586207


In [94]:
temp_mul = multiple_year_loan.groupby(multiple_year_loan['start_time'].dt.year).sum()['fraction_per_year']

In [95]:
temp_mul1 = temp_mul.rename_axis('start_time').reset_index(name = 'Mul_tot')

In [96]:
temp_mul1 # money of the loans correctly imputed for each year

,start_time,Mul_tot
0,2011,6.299118e+05
1,2012,9.101090e+06
2,2013,2.476658e+07
3,2014,2.954837e+07
4,2015,2.359988e+07
5,2016,2.074611e+07
6,2017,2.077317e+07
7,2018,4.785942e+06


In [97]:
same_year_loan.head() # needed to sum its results to the ones in temp_mul1

,loan_id,disburse_time,planned_expiration_time,duration,loan_amount
2,658010.0,2014-01-09,2014-02-15,37 days,400.0
3,659347.0,2014-01-17,2014-02-21,35 days,625.0
5,659605.0,2014-01-15,2014-02-20,36 days,350.0
6,660240.0,2014-01-20,2014-02-21,32 days,125.0
7,661601.0,2014-01-10,2014-02-25,46 days,1600.0


In [98]:
temp_sing = same_year_loan.groupby(same_year_loan['disburse_time'].dt.year).sum()['loan_amount']

In [99]:
temp_sing1 = temp_sing.rename_axis('disburse_time').reset_index(name = 'Sing_tot')

In [100]:
final_final = pd.merge(temp_mul1, temp_sing1, right_on = 'disburse_time', left_on = 'start_time', how = 'outer').fillna(0)

In [101]:
final_final['amount_per_year'] = final_final['Mul_tot'] + final_final['Sing_tot']

In [102]:
final_final[['start_time','amount_per_year']]

,start_time,amount_per_year
0,2011,6.299118e+05
1,2012,1.132391e+08
2,2013,1.246454e+08
3,2014,1.526974e+08
4,2015,1.593793e+08
5,2016,1.589528e+08
6,2017,1.720576e+08
7,2018,5.496367e+06
